In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("incident_event_log.csv")
df.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [2]:
df['opened_at'] = pd.to_datetime(df['opened_at'], errors='coerce')
df['resolved_at'] = pd.to_datetime(df['resolved_at'], errors='coerce')

C:\Users\FABIOLA PINTO Q\AppData\Local\Temp\ipykernel_8952\2824694668.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['opened_at'] = pd.to_datetime(df['opened_at'], errors='coerce')
C:\Users\FABIOLA PINTO Q\AppData\Local\Temp\ipykernel_8952\2824694668.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['resolved_at'] = pd.to_datetime(df['resolved_at'], errors='coerce')


Aquí lo que hicimos fue poner las columnas de apertura y resolución en formato fecha-hora real para poder ordenar, restar y medir duraciones sin problema. Como venían con formatos mezclados y algunos valores raros, lo que no se entiende se marca como faltante en vez de dañar el flujo.

In [3]:
df_final = pd.DataFrame()
df_final['time_to_resolve'] = (df['resolved_at'] - df['opened_at']).dt.total_seconds() / 3600.0
df_final['reassignment_count'] = df['reassignment_count']
df_final['sys_mod_count'] = df['sys_mod_count']
df_final['reopen_count'] = df['reopen_count']

Acá armamos el dataset final del modelo. Calculamos time_to_resolve en horas restando la fecha de resolución menos la de apertura (es la métrica clave del proceso) y copiamos los contadores operativos: reasignaciones, modificaciones del sistema y reaperturas. El resultado es un DataFrame compacto y enfocado solo en variables útiles para el modelado.

In [4]:
priority_dummies = pd.get_dummies(df['priority'], prefix='priority',dtype=int)
df_final = pd.concat([df_final, priority_dummies], axis=1)

Acá convertimos priority en columnas dummy y las pegamos al dataset. ¿Por qué? Porque vimos que priority pesa: el sistema la arma con impacto + urgencia, así que refleja qué tan crítico es cada ticket. Pasarla a priority_* (0/1) deja esa info lista para el modelo sin forzar un “orden” entre categorías y nos ayuda a ver cómo cada nivel afecta el tiempo de resolución.

In [9]:
df_final.replace("?", np.nan)
df_final.isna().sum()

time_to_resolve          0
reassignment_count       0
sys_mod_count            0
reopen_count             0
priority_1 - Critical    0
priority_2 - High        0
priority_3 - Moderate    0
priority_4 - Low         0
dtype: int64

Aquí lo que hicimos fue estandarizar los signos “?” como faltantes y luego contar cuántos NaN hay por columna. El resultado muestra que el único hueco real está en time_to_resolve (3141 filas), porque depende de tener apertura y resolución válidas. Con eso claro, esas filas se deben filtrar antes del modelado.

In [10]:
df_final = df_final.replace("?", np.nan)

In [11]:
df_final.dropna(inplace=True)

In [7]:
df_final.to_csv("incident_event_log_limpio.csv", index=False)

Con el DataFrame depurado y con las dummies listas, lo guardamos como un CSV nuevo en la misma carpeta. Esto deja una copia reproducible de los datos preparados para compartir, versionar en Git y usar directo en el notebook de modelado sin repetir toda la limpieza.